In [4]:
#
%load_ext autoreload
%autoreload 2
%matplotlib inline

import datetime as dtm
import matplotlib.dates as mpd
import pytz
tzutc = pytz.timezone('UTC')

#import operator
import math
import random
import numpy
import scipy
import scipy.optimize as spo
import itertools
import sys
#import scipy.optimize as spo
import os
import operator
#from PIL import Image as ipp
import multiprocessing as mpp
#
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
#import functools
#
#import shapely.geometry as sgp
#
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from geographiclib.geodesic import Geodesic as ggp
#
#
#import ANSStools as atp
from common import ANSStools as atp
#import bindex
import contours2kml
import globalETAS as gep
from eq_params import *
import roc_generic            # we'll eventually want to move to a new library of roc tools.
import random
#
#colors_ =  mpl.rcParams['axes.color_cycle']
colors_ = ['b', 'g', 'r', 'c', 'm', 'y', 'k']		# make sure these are correct...
#
emc = {'lon':32.13, 'lat':-115.30, 'event_date':dtm.datetime(2010,4,4,22,40,41, tzinfo=pytz.timezone('UTC'))}


In [5]:
def make_LD_figure(lats=[-31.], lons=[32.], lons=[-115., -113.],
                   to_dt=None, dt_0=30,
                   mc=2.5, d_lambda=1.76, dm=1.0, Ds=[1.0, 2.0]):
    '''
    # we want a liner density vs ETAS estimated linear densit figure.
    # first, get a catalog. find the biggest event in the catalog, plot linear distance from that event.
    # then, figure out the ETAS paramaters and see what we get and show for the domain 1. < D < 2.
    '''
    #
    #eqcat = atp.catfromANSS(lon=lons, lat=lats, minMag=cm, dates0=dates, Nmax=None, fout=None, rec_array=True)
    #eqcat.sort(order='event_date')
    # auto_cat(lon_center=None, lat_center=None, d_lat_0=.25, d_lon_0=.5, dt_0=10,  mc=2.5, mc_0=4.5, to_dt=None, catlen=5.0*365.0 range_factor=5., rec_array=True, **kwargs)
    eq_cat_params = atp.auto_cat_params(lat_center=emc['lat'], lon_center=emc['lon'],
                                        d_lat=.5, d_lon=.5, dt_0=dt_0, mc_0=5.0, 
                                        to_dt=to_dt)
    eqcat = atp.catfromANSS(lat=eq_cat_params['lat'], lon=eq_cat_params['lon'], minMag=mc, 
                            dates0=[eq_cat_params['mainshock_date'], eq_cat_params['mainshock_date'] + dtm.time_delta(days=cat_len_plus)],
                            fout=None, rec_array=True)
    #
    # find the biggest event; dump everything before that.
    #eqcat.sort(order='mag')   # ... except that we want the index too. so let's just spin it:
    mainshock = eqcat[0]
    mainshock_index = 0
    for j,rw in eqcat:
        if rw['mag']>mainshock['mag']: mainshock
        #
    #
    eqcat.sort(order='event_date')
    #
    # now, we need a distance formula. this can be using geographiclib or spherical dists. we could hook up some
    # of the class structures in the various etas codes...
    #dists = []
    #for j,ev in enumerate(eqcat):
    #    g1=ggp.WGS84.Inverse(self.mainshock['lat'], self.mainshock['lon'], rw['y'], rw['x'])
    #    r=g1['s12']/1000.)
    #
    dists = [[j,ggp.WGS84.Inverse(self.mainshock['lat'], self.mainshock['lon'], rw['y'], rw['x'])['s12']/1000.] for j,ev in enumerate(eqcat)]
    #
    dists2 =[[k,j,r] for k, (j,r) in enumerate(sorted(dists, key=lambda rw: rw[1]))]
    plt.figure()
    plt.plot(*zip(*[[r,k] for k,j,r in dists2]), '.-')
    